# October 31 - November 1 night time jet

In [ ]:
import xarray as xr
import altair as alt
import datetime
import pandas as pd 
import numpy as np
import glob

alt.data_transformers.disable_max_rows()

import sys
sys.path.append('../')
import sosutils

In [ ]:
dates = ['20221022', '20221023', '20221028', '20221029', '20221115', '20221116']
dates_i_want = ['20221022', '20221028', '20221115']
local_download_dir = 'sosnoqc'

# Download SoS data

In [ ]:
files = [sosutils.download_sos_data_day(date=d, local_download_dir=local_download_dir) for d in dates]
datasets = [xr.open_dataset(f) for f in files]

In [ ]:
sos_data = sosutils.merge_datasets_with_different_variables(datasets, dim='time')

In [ ]:
sos_data.to_dataframe().reset_index()['time'].min(), sos_data.to_dataframe().reset_index()['time'].max()

In [ ]:
[v for v in sos_data.data_vars if 'spd' in v and v.endswith('_d')]

# Identify variables we will use

In [ ]:
VARIABLE_NAMES = [
    'w_h2o__1m_d', # moisture flux (AKA latent heat flux), m^3/m^2/s
    'w_h2o__3m_d',
    'w_h2o__10m_d',
     
    'w_tc__1m_d', # sensible heat flux, K/m^2/s
    'w_tc__3m_d',
    'w_tc__10m_d',

    'Rsw_in_9m_d', # incoming shortwave radiation, W/m^2
    'Rsw_out_9m_d',


    'Tsoil_3_1cm_d', # soil temp    
    'Tsoil_8_1cm_d',
    'Tsoil_18_1cm_d',
    'Tsoil_28_1cm_d',

    'Tsoil_4_4cm_d',
    'Tsoil_9_4cm_d',
    'Tsoil_19_4cm_d',
    'Tsoil_29_4cm_d',

    'Tsoil_0_6cm_d', 
    'Tsoil_10_6cm_d',
    'Tsoil_20_6cm_d',
    'Tsoil_30_6cm_d',

    'Tsoil_1_9cm_d',
    'Tsoil_11_9cm_d',
    'Tsoil_21_9cm_d',
    'Tsoil_31_9cm_d',

    'Gsoil_d',  # heat flux at depth, W/m^2
    'Qsoil_d',  # soil moisture, % volume

    'T_1m_c',
    
    'tc_1m_d', # virtual temperature
    'tc_3m_d', 
    'tc_10m_d',

    # 'spd_1m_d', 'spd_3m_d', 'spd_10m_d' # wind speed
    'spd_1m_c', 'spd_2m_c', 'spd_3m_c', 'spd_5m_c', 'spd_10m_c', 'spd_15m_c', 'spd_20m_c',

    'Tcase_in_9m_d', 'Tcase_out_9m_d', # variables to calculate longwave radiation
    'Rpile_in_9m_d', 'Rpile_out_9m_d',
]

### Calculate longwave radiation calculation
https://www.eol.ucar.edu/content/calculation-long-wave-radiation

In [ ]:
STEFAN_BOLTZMANN_CONSTANT = 5.67e-8 # w/m^2/degK^4
sos_data['Rlw_in_9m_d'] = sos_data['Rpile_in_9m_d'] + STEFAN_BOLTZMANN_CONSTANT*(sos_data['Tcase_in_9m_d']+273.15)
sos_data['Rlw_out_9m_d'] = sos_data['Rpile_out_9m_d'] + STEFAN_BOLTZMANN_CONSTANT*(sos_data['Tcase_out_9m_d']+273.15)

VARIABLE_NAMES = VARIABLE_NAMES + ['Rlw_in_9m_d', 'Rlw_out_9m_d']

## Extract 5 minute-average data from the raw SoS dataset

In [ ]:
sos_data_df = sos_data[VARIABLE_NAMES].to_dataframe().reset_index().melt(id_vars='time', value_vars=VARIABLE_NAMES)
sos_data_df['height'] = sos_data_df['variable'].apply(sosutils.height_from_variable_name)
sos_data_df['tower'] = sos_data_df['variable'].apply(sosutils.tower_from_variable_name)
sos_data_df['measurement'] = sos_data_df['variable'].apply(sosutils.measurement_from_variable_name)
sos_data_df['time'] = sos_data_df['time'] - datetime.timedelta(hours=6)

In [ ]:
sos_data_df['day_string'] = sos_data_df['time'].dt.strftime('%Y%m%d')

In [ ]:
sos_data_df = sos_data_df[sos_data_df['day_string'].isin(dates_i_want)]

In [ ]:
sos_data_df.head()

# Examine diurnal cycles on different days

In [ ]:
# Add net shortwave radiation rows
net_sw_data = pd.merge(
    sos_data_df.query('variable == "Rsw_out_9m_d"'),
    sos_data_df.query('variable == "Rsw_in_9m_d"'),
    on = ['time', 'height', 'tower']
)[['time', 'height', 'tower', 'value_x', 'value_y']]

net_sw_data['measurement'] = 'shortwave radiation net'
net_sw_data['variable'] = 'Rsw_net_9m_d'
net_sw_data['value'] = net_sw_data['value_y'] - net_sw_data['value_x']

sos_data_df = pd.concat([
    sos_data_df,
    net_sw_data.drop(columns=['value_x', 'value_y'])
])

# Add net longwave radiation rows
net_lw_data = pd.merge(
    sos_data_df.query('variable == "Rlw_out_9m_d"'),
    sos_data_df.query('variable == "Rlw_in_9m_d"'),
    on = ['time', 'height', 'tower']
)[['time', 'height', 'tower', 'value_x', 'value_y']]

net_lw_data['measurement'] = 'longwave radiation net'
net_lw_data['variable'] = 'Rlw_net_9m_d'
net_lw_data['value'] = net_lw_data['value_y'] - net_lw_data['value_x']

sos_data_df = pd.concat([
    sos_data_df,
    net_lw_data.drop(columns=['value_x', 'value_y'])
])

In [ ]:
# Add time columns for easy plotting
# the individual day, as a string
sos_data_df['day'] = sos_data_df['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
sos_data_df['time_of_day'] = sos_data_df['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))

In [ ]:
# Create hourly soil temp dataset for plotting of vertical profiles
soil_temp_data = sos_data_df.query('measurement == "soil temperature"')
soil_temp_data = soil_temp_data.set_index(['time'])
soil_temp_data
soil_temp_data_1hr = soil_temp_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
soil_temp_data_1hr = soil_temp_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
soil_temp_data_1hr['day'] = soil_temp_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
soil_temp_data_1hr['time_of_day'] = soil_temp_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
soil_temp_data_1hr['hour'] = soil_temp_data_1hr['time'].dt.hour
soil_temp_data_1hr

In [ ]:
# Create hourly vertical heat flux dataset for plotting of vertical profiles
vert_heat_flux_data = sos_data_df.query('measurement == "w_tc_"')
vert_heat_flux_data = vert_heat_flux_data.set_index(['time'])
vert_heat_flux_data
vert_heat_flux_data_1hr = vert_heat_flux_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
vert_heat_flux_data_1hr = vert_heat_flux_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
vert_heat_flux_data_1hr['day'] = vert_heat_flux_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
vert_heat_flux_data_1hr['time_of_day'] = vert_heat_flux_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
vert_heat_flux_data_1hr['hour'] = vert_heat_flux_data_1hr['time'].dt.hour
vert_heat_flux_data_1hr

In [ ]:
# Create hourly vertical moisture flux dataset for plotting of vertical profiles
vert_moisture_flux_data = sos_data_df.query('measurement == "w_h2o_"')
vert_moisture_flux_data = vert_moisture_flux_data.set_index(['time'])
vert_moisture_flux_data
vert_moisture_flux_data_1hr = vert_moisture_flux_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
vert_moisture_flux_data_1hr = vert_moisture_flux_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
vert_moisture_flux_data_1hr['day'] = vert_moisture_flux_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
vert_moisture_flux_data_1hr['time_of_day'] = vert_moisture_flux_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
vert_moisture_flux_data_1hr['hour'] = vert_moisture_flux_data_1hr['time'].dt.hour
vert_moisture_flux_data_1hr

In [ ]:
# Create hourly potential temperature dataset for plotting of vertical profiles
potential_temp_data = sos_data_df.query('measurement == "potential temperature"')
potential_temp_data = potential_temp_data.set_index(['time'])
potential_temp_data
potential_temp_data_1hr = potential_temp_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
potential_temp_data_1hr = potential_temp_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
potential_temp_data_1hr['day'] = potential_temp_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
potential_temp_data_1hr['time_of_day'] = potential_temp_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
potential_temp_data_1hr['hour'] = potential_temp_data_1hr['time'].dt.hour
potential_temp_data_1hr

In [ ]:
# Create hourly potential temperature dataset for plotting of vertical profiles
wind_speed_data = sos_data_df.query('measurement == "wind speed"')
wind_speed_data = wind_speed_data.set_index(['time'])
wind_speed_data
wind_speed_data_1hr = wind_speed_data.groupby(['height', 'variable', 'tower', 'measurement', 'day']).resample('120Min').mean().drop(columns='height')
wind_speed_data_1hr = wind_speed_data_1hr.reset_index()
# Add time columns for easy plotting
# the individual day, as a string
wind_speed_data_1hr['day'] = wind_speed_data_1hr['time'].dt.strftime('%m/%d/%Y')
# the time of the day, assuming a random date
wind_speed_data_1hr['time_of_day'] = wind_speed_data_1hr['time'].apply(lambda dt: datetime.datetime(2022, 1, 1, dt.hour, dt.minute, dt.second))
wind_speed_data_1hr['hour'] = wind_speed_data_1hr['time'].dt.hour
wind_speed_data_1hr

In [ ]:
sos_data_df.variable.unique()

In [ ]:
sos_data_df.measurement.unique()

In [ ]:
time_series_plot = alt.Chart(sos_data_df).transform_filter(
    alt.FieldOneOfPredicate(
        'variable', 
        ['T_1m_c', 'Rsw_net_9m_d', 'Rlw_net_9m_d', 'w_h2o__1m_d', 'w_tc__1m_d', 'Qsoil_d']
    )
).mark_line().encode(
    alt.X('time_of_day:T'),
    alt.Y('value:Q'),
    alt.Color('day:N')
).properties(
    width=1000, 
    height = 150
).facet(
    row='variable'
).resolve_scale(y='independent', x='shared')

In [ ]:
bihourly_profiles_plot = (
    alt.Chart(soil_temp_data_1hr).mark_line().encode(
        alt.X(
            'value:Q', 
            sort='y', 
            # title='temp',
            scale=alt.Scale(domain=[-5, 15])
        ),
        alt.Y('height:Q', axis=alt.Axis(grid=False)),
        alt.Color('day:N')
    ).properties(
        width=73,
        height = 100
    ).facet(
        # row='day:N',
        column='hour:O',
        spacing=7
    ) & 
    alt.Chart(vert_heat_flux_data_1hr).mark_line().encode(
        alt.X(
            'value:Q', 
            sort='y', 
            title="w'tc'", 
            scale=alt.Scale(domain=[-0.1, 0.3])
        ),
        alt.Y('height:Q', axis=alt.Axis(grid=False)),
        alt.Color('day:N')
    ).properties(
        width=73,
        height = 100
    ).facet(
        column='hour:O',
        spacing=7
    ) & 
    alt.Chart(vert_moisture_flux_data_1hr).mark_line().encode(
        alt.X(
            'value:Q', 
            sort='y', 
            title="w'q'", 
            scale=alt.Scale(domain=[-0.005, 0.025])
        ),
        alt.Y('height:Q', axis=alt.Axis(grid=False)),
        alt.Color('day:N')
    ).properties(
        width=73,
        height = 100
    ).facet(
        column='hour:O',
        spacing=7
    ) &  
    alt.Chart(potential_temp_data_1hr).mark_line().encode(
        alt.X(
            'value:Q', 
            sort='y', 
            title="tc", 
        ),
        alt.Y('height:Q', axis=alt.Axis(grid=False)),
        alt.Color('day:N')
    ).properties(
        width=73,
        height = 100
    ).facet(
        column='hour:O',
        spacing=7
    ) &  
    alt.Chart(wind_speed_data_1hr).mark_line().encode(
        alt.X(
            'value:Q', 
            sort='y', 
            title="wind speed", 
        ),
        alt.Y('height:Q', axis=alt.Axis(grid=False)),
        alt.Color('day:N')
    ).properties(
        width=73,
        height = 100
    ).facet(
        column='hour:O',
        spacing=7
    )
)

In [ ]:
time_series_plot & bihourly_profiles_plot

* √ convert to mtn time
* winds with height too
* include net longwave